In [50]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline

from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

In [51]:
df_train = pd.read_csv("data-q1/train.csv")
df_test = pd.read_csv("data-q1/test.csv")

df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

df_test['sales'] = np.nan

df_train['type'] = "train"
df_test['type'] = "test"

In [52]:
df_train.tail()

,id,date,seller,item,sales,type
899995,899995,2020-04-03,6,50,11,train
899996,899996,2020-04-03,7,50,6,train
899997,899997,2020-04-03,8,50,25,train
899998,899998,2020-04-03,9,50,17,train
899999,899999,2020-04-03,10,50,16,train


In [53]:
df_test.head()

,id,date,seller,item,sales,type
0,0,2020-04-04,1,1,NaN,test
1,1,2020-04-04,2,1,NaN,test
2,2,2020-04-04,3,1,NaN,test
3,3,2020-04-04,4,1,NaN,test
4,4,2020-04-04,5,1,NaN,test


In [54]:
bigdata = pd.concat([df_train,df_test], ignore_index=True, sort=False)

In [55]:
bigdata.head()

,id,date,seller,item,sales,type
0,0,2015-05-01,1,1,21.0,train
1,1,2015-05-01,2,1,45.0,train
2,2,2015-05-01,3,1,28.0,train
3,3,2015-05-01,4,1,34.0,train
4,4,2015-05-01,5,1,21.0,train


In [56]:
bigdata = bigdata.sort_values(['seller', 'item' , 'date'], ascending=[True, True,True])

In [57]:
bigdata['date'] = pd.to_datetime(bigdata['date'])

bigdata['week'] = bigdata['date'].dt.isocalendar().week
bigdata['day'] = bigdata['date'].dt.isocalendar().day
bigdata['year'] = bigdata['date'].dt.isocalendar().year
bigdata['month'] = bigdata['date'].dt.month

In [58]:
pd.get_dummies(bigdata['seller'], prefix='seller',drop_first=True)
# pd.get_dummies(bigdata['item'], prefix='item',drop_first=True)

bigdata = pd.concat([bigdata,pd.get_dummies(bigdata['item'], prefix='item')],axis=1)
# bigdata.drop(['item'],axis=1, inplace=True)

In [59]:
gps = bigdata.groupby(['year','seller','item'])['sales'].mean().to_frame()

dict_sales = {}
for (year,seller,item),sales in gps.iterrows():
    dict_sales[ f"{year}-{seller}-{item}"] = round(sales[0],0) 
    
bigdata['mean_year_sales'] = bigdata.apply(lambda x : dict_sales[f"{x['year']}-{x['seller']}-{x['item']}"], axis=1)

In [60]:
gps = bigdata.groupby(['year','seller','item'])['sales'].var().to_frame()

dict_sales = {}
for (year,seller,item),sales in gps.iterrows():
    dict_sales[ f"{year}-{seller}-{item}"] = round(sales[0],0) 
    
bigdata['var_year_sales'] = bigdata.apply(lambda x : dict_sales[f"{x['year']}-{x['seller']}-{x['item']}"], axis=1)

In [77]:
X = bigdata[ bigdata['type'] == 'train'].drop(['seller','item','date','type','id','sales'],axis=1)

In [78]:
X = X.values
y = bigdata[ bigdata['type'] == 'train' ]['sales'].tolist()

In [79]:
estimators = [
    ('GB', GradientBoostingRegressor(random_state=0)),
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,  random_state=42)
)

# reg = RandomForestRegressor(n_estimators=10,  random_state=42)

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=42)

In [ ]:
reg.fit(X_train, y_train)

In [67]:
y_pred = reg.predict(X_test)

In [68]:
X = bigdata[ bigdata['type'] == 'test'].drop(['seller','item','date','type','id','sales'],axis=1)

In [69]:
y_pred = reg.predict(X)

In [70]:
bigdata.loc[ bigdata['type'] == 'test' , 'sales'] = y_pred

In [71]:
output = bigdata[ bigdata['type'] == 'test' ]
output = output.sort_values(['id'], ascending=[True])
output.drop(['date','seller','item','type'] , inplace=True,axis=1)

In [72]:
output.drop(output.columns.difference(['id','sales']), 1, inplace=True)

In [73]:
output.to_csv('output_q1_11.csv',index=False)

In [37]:
from sklearn.metrics import mean_squared_error

In [38]:
mean_squared_error(y_pred,y_test)

69.05775632386992

In [ ]:
DM.fit(X_train, Y_train)
preds = DM.predict(X_test)
print('- tfidf: ', DM.steps[1][0], ': accuracy' , accuracy_score(preds, Y_test))
print('- tfidf: ', DM.steps[1][0], ': RECALL ' , recall_score(preds, Y_test))
print('- tfidf: ', DM.steps[1][0], ': F1_SCORE' , f1_score(preds, Y_test))

In [18]:
sales = bigdata.sales.values.reshape(-1,1826)

In [48]:
results = []
y_actual = []
preds = []
for item in sales :
    X = np.arange(1826)
    y = item
    
    X_train = X[:1800]
    y_train = y[:1800]
    X_test = X[1800:]
    y_test = y[1800:]

    fittedParameters = np.polyfit(X_train, y_train, 1)
    modelPredictions = np.polyval(fittedParameters, X_test)
#     y_actual.extend(y_test.tolist())
#     preds.extend(modelPredictions.tolist())
    res = y_train.tolist()
    res.extend(modelPredictions.tolist())
    results.append(res)

results = np.array(results)
# absError = modelPredictions - y_test
# SE = np.square(absError) # squared errors
# MSE = np.mean(SE) # mean squared errors
# RMSE = np.sqrt(MSE) # Root Mean Squared Error, RMSE
# Rsquared = 1.0 - (np.var(absError) / np.var(y_test))
# print('RMSE:', RMSE)
# print('R-squared:', Rsquared)

In [52]:
results = results.reshape(-1,1)

In [53]:
results.shape

(913000, 1)

In [54]:
bigdata.sales = results

In [55]:
bigdata.head(1826)

,id,date,seller,item,sales,type
0,0,2015-05-01,1,1,21.000000,train
10,10,2015-05-02,1,1,26.000000,train
20,20,2015-05-03,1,1,25.000000,train
30,30,2015-05-04,1,1,26.000000,train
40,40,2015-05-05,1,1,28.000000,train
...,...,...,...,...,...,...
900210,210,2020-04-25,1,1,56.104536,test
900220,220,2020-04-26,1,1,56.114815,test
900230,230,2020-04-27,1,1,56.125094,test
900240,240,2020-04-28,1,1,56.135374,test


In [57]:
bgbg = bigdata.groupby(['seller','item']).apply(lambda group: group.interpolate(method='linear'))

In [59]:
bgbg.head(1826)

,id,date,seller,item,sales,type
0,0,2015-05-01,1,1,21.0,train
10,10,2015-05-02,1,1,26.0,train
20,20,2015-05-03,1,1,25.0,train
30,30,2015-05-04,1,1,26.0,train
40,40,2015-05-05,1,1,28.0,train
...,...,...,...,...,...,...
900210,210,2020-04-25,1,1,35.0,test
900220,220,2020-04-26,1,1,35.0,test
900230,230,2020-04-27,1,1,35.0,test
900240,240,2020-04-28,1,1,35.0,test


In [57]:
output = bgbg[ bgbg['type'] == 'test' ]

In [58]:
output = bgbg[ bgbg['type'] == 'test' ]
output = output.sort_values(['id'], ascending=[True])
output.drop(['date','seller','item','type'] , inplace=True,axis=1)

In [60]:
output.to_csv('output_q1_2.csv',index=False)

In [59]:
output

,id,sales
900000,0,55.888672
900001,1,79.329190
900002,2,70.275957
900003,3,64.818304
900004,4,47.048200
...,...,...
912995,12995,19.942550
912996,12996,18.785890
912997,12997,32.569134
912998,12998,27.793415
